<a href="https://colab.research.google.com/github/Shivam10816/CS22M082_ASSIGNEMNT_1/blob/q2/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()
train_data = np.reshape(train_data,(len(train_data),train_data.shape[1]**2))
test_data = np.reshape(test_data,(len(test_data),test_data.shape[1]**2))
test_labels =np.reshape(test_labels,(1,len(test_data)))


In [ ]:
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import numpy as np
import random
from keras.datasets import fashion_mnist

(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()
train_data = np.reshape(train_data/255,(len(train_data),train_data.shape[1]**2))
test_data = np.reshape(test_data/255,(len(test_data),test_data.shape[1]**2))
test_labels =np.reshape(test_labels,(1,len(test_data)))
print(np.max(train_data))
class neural_network:

  #it initializes W and b
  def __init__(self,train_data,train_label,hi):
    
    self.train_data=train_data
    self.train_label=train_label
    l= train_data.shape[1]

    self.W =[self.xavier_init(hi[0],l)] 
    self.b =[self.xavier_init(1,hi[0])]
    for i in range(1,len(hi)) :
      self.W.append(self.xavier_init(hi[i],hi[i-1]))
      self.b.append(self.xavier_init(1,hi[i])) 
    self.W.append(self.xavier_init(10,hi[-1]))
    
    self.b.append(self.xavier_init(1,10))
  
  def WX_plus_B(self,W, X, b):
    
    result = np.dot(X, W.transpose())
    row_count = result.shape[0]
    
    row_matrix_repeated = np.tile(b, (row_count, 1))
    return result + row_matrix_repeated

    
  def xavier_init(self,input_size, output_size):
    """
    Initialize weights using Xavier initialization.

    Parameters:
    input_size (int): number of input units.
    output_size (int): number of output units.

    Returns:
    weights (ndarray): array of shape (input_size, output_size) containing the initialized weights.
    """
    # Calculate the variance of the weights
    variance = 2.0 / (input_size + output_size)

    # Calculate the standard deviation of the weights
    standard_deviation = np.sqrt(variance)

    # Generate random weights from a normal distribution with mean 0 and standard deviation standard_deviation
    weights = np.random.normal(loc=0, scale=standard_deviation, size=(input_size, output_size))

    return weights

  #calculates sigmoid for matrix
  def sigmoid(self,x):
  
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
  
  def sum_columns(self,matrix):
    if isinstance(matrix, np.ndarray):
        # if matrix is a numpy array, convert it to a list
        matrix = matrix.tolist()
    
    # sum the elements of each column and store in a list
    column_sums = [sum(col) for col in zip(*matrix)]
    
    # convert the list to a 2D matrix of shape (1 x n)
    row_matrix = np.array([column_sums])
    
    return row_matrix
  #softmax for matrix
  def softmax(self,x):
    # Subtract the maximum value in each row from all the values in that row
    # to prevent numerical instability from very large or very small values
    # in the exponentials of the softmax function.
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / np.sum(e_x, axis=1, keepdims=True)
  
  def subtract_matrices(self,W, W_theta, step_size):
    """
    Subtract the matrices in the second list from the matrices in the first list, after multiplying the matrices in the
    second list by a step size.

    Args:
        first_list (list): A list of numpy arrays representing the first set of matrices.
        second_list (list): A list of numpy arrays representing the second set of matrices.
        step_size (float): The step size to multiply the second set of matrices by.

    Returns:
        list: A list of numpy arrays representing the result of subtracting the second set of matrices from the first set
        of matrices after multiplying the second set of matrices by the step size.
    """
    result_list = []
    for i in range(len(self.W)):
        result = W[i] - step_size * (W_theta[i]/self.batch_size)
        result_list.append(result)
    return result_list

  def sigmoid_derivative(self,matrix):
    """
    Calculate the derivative of the sigmoid function on a 2D matrix.

    Args:
        matrix (numpy.ndarray): A numpy array representing the matrix.

    Returns:
        numpy.ndarray: A numpy array representing the result of calculating the sigmoid derivative on the matrix.
    """
    shift = np.max(matrix, axis=1, keepdims=True)
    exp_matrix = np.exp(matrix - shift)
    sig = 1 / (1 + exp_matrix)
    return sig * (1 - sig)

  def forward_pro(self,X):
    A=[]
    H=[]
    A.append(self.WX_plus_B(self.W[0],X,self.b[0])) # a0 = WoX +bo

    for i in range(1,len(hi)):

      H.append(self.sigmoid(A[-1])) # hi = g(ai)
      #print(H[i-1])
      A.append( self.WX_plus_B(self.W[i],H[-1],self.b[i])) # ai = WiX +bi

    H.append(self.sigmoid(A[-1]))
    A.append(self.WX_plus_B(self.W[-1],H[-1],self.b[-1]))
    
    y_hat = self.softmax((A[-1]))
    
    
    
    return A,H,y_hat

  def back_prop(self,X,Y,A,H,y_hat):
    W_theta , b_theta,H_theta,A_theta =self.W,self.b,H,A
    #print(np.argmax(y_hat),lable)
    ey = np.zeros((y_hat.shape[0],y_hat.shape[1]))

    for i in range(0,len(Y)):
      ey[i][Y[i]]=1
  
    L =len(A_theta)
    A_theta[L-1]=(-(ey-y_hat))
    
    #-------------------------
    for k in range(L-1,0,-1):
      
      W_theta[k]=((np.matmul(A_theta[k].transpose(),H[k-1]))) # athetak*h[k-1]
      b_theta[k]=self.sum_columns(A_theta[k])
      H_theta[k-1]=np.matmul(A_theta[k],self.W[k])
  
      A_theta[k-1]=H_theta[k-1]*self.sigmoid_derivative(A[k-1])

    W_theta[0] = ((np.matmul(A_theta[0].transpose(),X)))
    b_theta[0]=self.sum_columns(A_theta[0])

    # print("-------W-----")
    # for i in W_theta :
    #   print(i)
    # print("-------b-----")
    # for i in b_theta :
    #   print(i)
    # print("-------A-----")
    # for i in A_theta :
    #   print(i)
    # print("-------H-----")
    # for i in H_theta :
    #   print(i)
    
    return W_theta , b_theta

  def accuracy(self, X_test, y_test):
    
    # Feed forward through the network
    A,H,y_hat =self.forward_pro(X_test)
    
    
    y_pred = np.argmax(y_hat, axis=1)
    print(y_pred.shape)
    # Calculate accuracy
    acc = np.mean(y_pred == y_test)
    # Calculate accuracy
    

    return acc

  def sgd(self,step_size,batch_size,epoch):
    N = int(len(self.train_data)/batch_size)
    self.batch_size = batch_size
    for e in range(0,epoch):
      start_time = time.time()
      for k in range(0,N):
          minibatch = self.train_data[k*batch_size:min(k*batch_size+batch_size,len(self.train_data))]
          minibatch_lable=self.train_label[k*batch_size:min(k*batch_size+batch_size,len(self.train_data))]
          
          A,H,y_hat=self.forward_pro(minibatch)

          dW,db = self.back_prop(minibatch,minibatch_lable,A,H,y_hat)
          # print(np.min(dW[0]),np.min(dW[1]),np.min(dW[2]))
          # print(np.min(db[0]),np.min(db[1]),np.min(db[2]))
          # print(np.max(dW[0]),np.max(dW[1]),np.max(dW[2]))
          # print(np.max(db[0]),np.max(db[1]),np.max(db[2]))
          # print("\n\n\n")
          self.b =self.subtract_matrices(self.b,db,step_size)
          self.W =self.subtract_matrices(self.W,dW,step_size)
          print(np.min(Net.W[0]),np.min(Net.W[1]),np.min(Net.W[2]))
          print(np.max(Net.W[0]),np.max(Net.W[1]),np.max(Net.W[2]))
          print(np.min(Net.b[0]),np.min(Net.b[1]),np.min(Net.b[2]))

          print(np.max(Net.b[0]),np.max(Net.b[1]),np.max(Net.b[2]))
          print("\n\n\n")
          
      print("epoch ",e ,"  %s seconds " % (time.time() - start_time))
     
      
hi=[10,10,10]   
Net = neural_network(train_data,train_labels,hi)

print("accuracy :-",Net.accuracy(test_data,test_labels)*100,"%")
Net.sgd(0.00001,16,10)
print("accuracy :-",Net.accuracy(test_data,test_labels)*100,"%")


Streaming output truncated to the last 5000 lines.
481332.1220187982 1387.5616327734374 74.4999534375




-1.2972042879081545e-30 0.0 0.0
37662.27672147901 462.24971109375 42.999973125
0.0 0.0 0.0
53418.732238271485 462.24971109375 42.999973125




-3.051278574680883e-19 0.0 0.0
113575.32354662841 812.24949234375 56.999964375
0.0 0.0 0.0
164937.41253905275 812.24949234375 56.999964375




0.0 0.0 0.0
112312.40156098579 826.5619833984375 57.4999640625
0.0 0.0 0.0
170801.28485069275 826.5619833984375 57.4999640625




0.0 0.0 0.0
63066.74979894301 624.999609375 49.99996875
0.0 0.0 0.0
97656.18896484375 624.999609375 49.99996875




0.0 0.0 0.0
192891.1325334999 1040.0618499609375 64.4999596875
0.0 0.0 0.0
270432.33195624937 1040.0618499609375 64.4999596875




0.0 0.0 0.0
165281.53297348713 960.999399375 61.99996125
0.0 0.0 0.0
230880.10569984376 960.999399375 61.99996125




0.0 0.0 0.0
197443.94426174997 1072.5618296484374 65.4999590625
0.0 0.0 0.0
287597.39935307554 1072.5618296484374

In [67]:
print(1e-12)

1e-12


In [ ]:


b =[np.ones(1,50)]

# Net.sgd(0.0001,16,100)
#Net.calculate_accuracy(test_data,test_labels)

In [ ]:
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()
train_data = np.reshape(train_data/255,(len(train_data),train_data.shape[1]**2))
test_data = np.reshape(test_data/255,(len(test_data),test_data.shape[1]**2))
test_labels =np.reshape(test_labels,(1,len(test_data)))


In [ ]:
print(test_data.shape)
print(test_labels.shape)